# LLaMA and Llama-2 in TransformerLens

## Setup (skip)

In [2]:
# NBVAL_IGNORE_OUTPUT
# Janky code to do different setup when run in a Colab notebook vs VSCode
import os

DEVELOPMENT_MODE = False
IN_VSCODE = False
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")
    
%pip install transformers==4.31.0 # Llama requires transformers>=4.31.0 and transformers in turn requires Python 3.8
%pip install sentencepiece # Llama tokenizer requires sentencepiece
    
if IN_GITHUB or IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis

Running as a Colab notebook
  Cloning https://github.com/coolvision/TransformerLens.git (to revision llama_4bit_v2) to /tmp/pip-req-build-lpt2rmoh
  Running command git clone --filter=blob:none --quiet https://github.com/coolvision/TransformerLens.git /tmp/pip-req-build-lpt2rmoh
  Running command git checkout -b llama_4bit_v2 --track origin/llama_4bit_v2
  Switched to a new branch 'llama_4bit_v2'
  Branch 'llama_4bit_v2' set up to track remote branch 'llama_4bit_v2' from 'origin'.
  Resolved https://github.com/coolvision/TransformerLens.git to commit b2b80cb92f4aa6d63a456196f0c3472b3d34c6eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
elif IN_VSCODE:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv

Using renderer: colab


In [4]:
# Import stuff
import torch
import tqdm.auto as tqdm
import plotly.express as px

from transformers import LlamaForCausalLM, LlamaTokenizer
from tqdm import tqdm
from jaxtyping import Float

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Loading LLaMA

LLaMA weights are not available on HuggingFace, so you'll need to download and convert them
manually:

1. Get LLaMA weights here: https://docs.google.com/forms/d/e/1FAIpQLSfqNECQnMkycAp2jP4Z9TFX0cGR4uf7b_fBxjY_OjhJILlKGA/viewform

2. Convert the official weights to huggingface:

```bash
python src/transformers/models/llama/convert_llama_weights_to_hf.py \
    --input_dir /path/to/downloaded/llama/weights \
    --model_size 7B \
    --output_dir /llama/weights/directory/
```

Note: this didn't work for Arthur by default (even though HF doesn't seem to show this anywhere). I
had to change <a
href="https://github.com/huggingface/transformers/blob/07360b6/src/transformers/models/llama/convert_llama_weights_to_hf.py#L295">this</a>
line of my pip installed `src/transformers/models/llama/convert_llama_weights_to_hf.py` file (which
was found at
`/opt/conda/envs/arthurenv/lib/python3.10/site-packages/transformers/models/llama/convert_llama_weights_to_hf.py`)
from `input_base_path=os.path.join(args.input_dir, args.model_size),` to `input_base_path=os.path.join(args.input_dir),`

3. Change the ```MODEL_PATH``` variable in the cell below to where the converted weights are stored.

In [5]:
MODEL_PATH=''

if MODEL_PATH:
    tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
    hf_model = LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True)

    model = HookedTransformer.from_pretrained("llama-7b", hf_model=hf_model, device="cpu", fold_ln=False, center_writing_weights=False, center_unembed=False, tokenizer=tokenizer)

    model = model.to("cuda" if torch.cuda.is_available() else "cpu")
    model.generate("The capital of Germany is", max_new_tokens=20, temperature=0)

## Loading LLaMA-2
LLaMA-2 is hosted on HuggingFace, but gated by login.

Before running the notebook, log in to HuggingFace via the cli on your machine:
```bash
transformers-cli login
```
This will cache your HuggingFace credentials, and enable you to download LLaMA-2.

## Install additional dependenceis requred for quantization

In [6]:
%pip install bitsandbytes==0.42.0
%pip install accelerate

## Load quantized model

In [7]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

LLAMA_2_7B_CHAT_PATH = "meta-llama/Llama-2-7b-chat-hf"
inference_dtype = torch.float32
# inference_dtype = torch.float32
# inference_dtype = torch.float16

hf_model = AutoModelForCausalLM.from_pretrained(LLAMA_2_7B_CHAT_PATH,
                                             torch_dtype=inference_dtype,
                                             device_map = "cuda:0",
                                             quantization_config=BitsAndBytesConfig(load_in_4bit=True))

tokenizer = AutoTokenizer.from_pretrained(LLAMA_2_7B_CHAT_PATH)

model = HookedTransformer.from_pretrained(LLAMA_2_7B_CHAT_PATH,
                                             hf_model=hf_model,
                                             dtype=inference_dtype,
                                             fold_ln=False,
                                             fold_value_biases=False,
                                             center_writing_weights=False,
                                             center_unembed=False,
                                             tokenizer=tokenizer)

model.generate("The capital of Germany is", max_new_tokens=2, temperature=0)



model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer


  0%|          | 0/2 [00:00<?, ?it/s]

'The capital of Germany is Berlin.'

### Verify GPU memory use

In [12]:
print("free(Gb):", torch.cuda.mem_get_info()[0]/1000000000, "total(Gb):", torch.cuda.mem_get_info()[1]/1000000000)

free(Gb): 9.29988608 total(Gb): 15.835660288


### Compare logits with HuggingFace model

In [14]:
prompts = [
    "The capital of Germany is",
    "2 * 42 = ",
    "My favorite",
    "aosetuhaosuh aostud aoestuaoentsudhasuh aos tasat naostutshaosuhtnaoe usaho uaotsnhuaosntuhaosntu haouaoshat u saotheu saonuh aoesntuhaosut aosu thaosu thaoustaho usaothusaothuao sutao sutaotduaoetudet uaosthuao uaostuaoeu aostouhsaonh aosnthuaoscnuhaoshkbaoesnit haosuhaoe uasotehusntaosn.p.uo ksoentudhao ustahoeuaso usant.hsa otuhaotsi aostuhs",
]

model.eval()
hf_model.eval()
prompt_ids = [tokenizer.encode(prompt, return_tensors="pt") for prompt in prompts]
tl_logits = [model(prompt_ids).detach().cpu() for prompt_ids in tqdm(prompt_ids)]

# hf logits are really slow as it's on CPU. If you have a big/multi-GPU machine, run `hf_model = hf_model.to("cuda")` to speed this up
logits = [hf_model(prompt_ids).logits.detach().cpu() for prompt_ids in tqdm(prompt_ids)]

for i in range(len(prompts)):
    if i == 0:
        print("logits[i]", i, logits[i].dtype, logits[i])
        print("tl_logits[i]", i, tl_logits[i].dtype, tl_logits[i])
    assert torch.allclose(logits[i], tl_logits[i], atol=1e-4, rtol=1e-2)

100%|██████████| 4/4 [00:02<00:00,  1.78it/s]


logits[i] 0 torch.float32 tensor([[[ 2.6141e-01, -1.3572e+00, -2.5338e-03,  ...,  1.5789e+00,
           1.7533e+00,  6.7760e-01],
         [-9.6479e+00, -5.7782e+00, -4.9753e+00,  ..., -6.4692e+00,
          -6.1699e+00, -6.2673e+00],
         [-7.2656e+00, -5.5785e+00,  1.7281e+00,  ..., -1.5952e+00,
          -6.6061e+00, -3.6781e+00],
         [-1.4183e+00, -1.1532e-01,  3.3884e+00,  ..., -2.7672e+00,
          -1.5437e+00, -2.0235e+00],
         [-1.5692e-01, -4.4547e-02,  1.2237e+01,  ...,  4.0257e-01,
          -5.5462e-01,  6.4535e-01],
         [-4.0320e+00, -3.9415e+00,  7.8710e+00,  ..., -1.0899e+00,
          -3.5164e+00, -1.2452e+00]]])
tl_logits[i] 0 torch.float32 tensor([[[ 2.6141e-01, -1.3572e+00, -2.5325e-03,  ...,  1.5789e+00,
           1.7533e+00,  6.7759e-01],
         [-9.6479e+00, -5.7782e+00, -4.9753e+00,  ..., -6.4692e+00,
          -6.1700e+00, -6.2673e+00],
         [-7.2656e+00, -5.5785e+00,  1.7281e+00,  ..., -1.5952e+00,
          -6.6061e+00, -3.6781e+00]

## TransformerLens Demo

### Reading from hooks

In [15]:
llama_text = "Natural language processing tasks, such as question answering, machine translation, reading comprehension, and summarization, are typically approached with supervised learning on taskspecific datasets."
llama_tokens = model.to_tokens(llama_text)
llama_logits, llama_cache = model.run_with_cache(llama_tokens, remove_batch_dim=True)

attention_pattern = llama_cache["pattern", 0, "attn"]
llama_str_tokens = model.to_str_tokens(llama_text)

print("Layer 0 Head Attention Patterns:")
display(cv.attention.attention_patterns(tokens=llama_str_tokens, attention=attention_pattern))

Layer 0 Head Attention Patterns:


### Writing to hooks

In [16]:
layer_to_ablate = 0
head_index_to_ablate = 31

# We define a head ablation hook
# The type annotations are NOT necessary, they're just a useful guide to the reader
#
def head_ablation_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    print(f"Shape of the value tensor: {value.shape}")
    value[:, :, head_index_to_ablate, :] = 0.
    return value

original_loss = model(llama_tokens, return_type="loss")
ablated_loss = model.run_with_hooks(
    llama_tokens,
    return_type="loss",
    fwd_hooks=[(
        utils.get_act_name("v", layer_to_ablate),
        head_ablation_hook
        )]
    )
print(f"Original Loss: {original_loss.item():.3f}")
print(f"Ablated Loss: {ablated_loss.item():.3f}")

Shape of the value tensor: torch.Size([1, 34, 32, 128])
Original Loss: 2.841
Ablated Loss: 2.806
